In [8]:
import importlib
import sys
import os

# Add scripts path if not already added
scripts_dir = os.path.abspath(os.path.join(os.getcwd(), "../scripts"))
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

# Force reload the module in case it was cached
import data_utils
importlib.reload(data_utils)

from data_utils import transform_ts_data_into_features_and_target_loop


In [9]:
# Load time series data
ts_df = pd.read_csv("../data/ts_data.csv", parse_dates=["pickup_hour"])
ts_df.set_index("pickup_hour", inplace=True)
print("✅ Time series data shape:", ts_df.shape)
ts_df.head()


✅ Time series data shape: (8748, 205)


,4074.14,4298.05,4461.07,4762.05,4977.03,4993.02,4993.15,5024.10,5033.01,5105.01,...,JC102,JC103,JC104,JC105,JC107,JC108,JC109,JC110,JC115,JC116
pickup_hour,,,,,,,,,,,,,,,,,,,,,
2024-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,2,0,3,11
2024-01-01 01:00:00,0,0,0,0,0,0,0,0,0,0,...,4,0,2,5,0,0,4,1,4,2
2024-01-01 02:00:00,0,0,0,0,0,0,0,0,0,0,...,2,2,4,1,0,0,2,1,6,4
2024-01-01 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,2,2,1,0,0,1,2,2,0
2024-01-01 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,4,0


In [10]:
# Define top 3 start station IDs
top_location_ids = ["HB102", "JC115", "HB105"]


In [11]:
# Generate features using utility function
feature_dfs = transform_ts_data_into_features_and_target_loop(ts_df, top_location_ids)

# Show one example
feature_dfs[top_location_ids[0]].head()



📍 Processing location: HB102
📍 Processing location: JC115
📍 Processing location: HB105


,target,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,...,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28
pickup_hour,,,,,,,,,,,,,,,,,,,,,
2024-01-02 04:00:00,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,3.0,4.0,...,1.0,0.0,0.0,0.0,0.0,2.0,2.0,20.0,5.0,5.0
2024-01-02 05:00:00,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,3.0,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,20.0,5.0
2024-01-02 06:00:00,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,5.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,20.0
2024-01-02 07:00:00,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,5.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0
2024-01-02 08:00:00,4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,5.0,5.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0


In [12]:
# Save each location's features DataFrame to a CSV
for location_id in top_location_ids:
    output_path = f"../data/features/{location_id}.csv"
    feature_dfs[location_id].to_csv(output_path)
    print(f"✅ Saved features for location {location_id} to {output_path}")


✅ Saved features for location HB102 to ../data/features/HB102.csv
✅ Saved features for location JC115 to ../data/features/JC115.csv
✅ Saved features for location HB105 to ../data/features/HB105.csv


In [13]:
!pip install hopsworks --quiet
import hopsworks



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [14]:
project = hopsworks.login()
fs = project.get_feature_store()


2025-05-03 12:16:27,602 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-03 12:16:27,611 INFO: Initializing external client
2025-05-03 12:16:27,612 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 12:16:28,598 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691


In [15]:
import hopsworks
import pandas as pd

# Combine the 3 DataFrames into one with a 'location_id' column
combined_df = pd.concat(
    [df.assign(location_id=loc_id) for loc_id, df in feature_dfs.items()]
).reset_index()

print("✅ Combined dataframe shape:", combined_df.shape)
combined_df.head()


✅ Combined dataframe shape: (26160, 31)


,pickup_hour,target,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,...,lag_20,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28,location_id
0,2024-01-02 04:00:00,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,3.0,...,0.0,0.0,0.0,0.0,2.0,2.0,20.0,5.0,5.0,HB102
1,2024-01-02 05:00:00,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,2.0,2.0,20.0,5.0,HB102
2,2024-01-02 06:00:00,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,20.0,HB102
3,2024-01-02 07:00:00,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,5.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,HB102
4,2024-01-02 08:00:00,4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,HB102


In [16]:
project = hopsworks.login()
fs = project.get_feature_store()


2025-05-03 12:16:34,254 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-03 12:16:34,261 INFO: Initializing external client
2025-05-03 12:16:34,261 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 12:16:35,053 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215691


In [17]:
# Create combined_df with pickup_hour as a column
combined_df = pd.concat([
    df.assign(location_id=loc_id) for loc_id, df in feature_dfs.items()
]).reset_index()  # pickup_hour is moved to column here


In [18]:
# Convert pickup_hour to datetime and location_id to string
combined_df["pickup_hour"] = pd.to_datetime(combined_df["pickup_hour"])
combined_df["location_id"] = combined_df["location_id"].astype(str)

# Ensure all other columns are float
combined_df = combined_df.astype({
    col: "float" for col in combined_df.columns if col not in ["pickup_hour", "location_id"]
})


In [19]:
from hsfs.feature import Feature
from hsfs.feature_group import FeatureGroup

feature_group = fs.get_or_create_feature_group(
    name="citi_bike_features_group",
    version=1,
    description="Lag features for top 3 Citi Bike locations",
    primary_key=["pickup_hour", "location_id"],
    event_time="pickup_hour",
    features=[
        Feature(name=col, type="double") if col not in ["pickup_hour", "location_id"]
        else Feature(name=col, type="string") if col == "location_id"
        else Feature(name=col, type="timestamp")
        for col in combined_df.columns
    ]
)


In [20]:
feature_group.insert(combined_df, write_options={"wait_for_job": True})
print("✅ Feature group successfully written to Hopsworks!")


Uploading Dataframe: 100.00% |██████████| Rows 26160/26160 | Elapsed Time: 00:09 | Remaining Time: 00:00


Launching job: citi_bike_features_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215691/jobs/named/citi_bike_features_group_1_offline_fg_materialization/executions
2025-05-03 12:17:23,516 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-05-03 12:18:41,546 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-05-03 12:20:33,819 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-05-03 12:20:36,942 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-05-03 12:20:37,039 INFO: Waiting for log aggregation to finish.
2025-05-03 12:20:51,940 INFO: Execution finished successfully.
✅ Feature group successfully written to Hopsworks!
